In [89]:
import pandas as pd
df = pd.read_excel("Input.xlsx")

In [1]:
import os
stop_words_dir = "E:\Projects\Article Text NLP Analysis\StopWords"

stop_word_files = os.listdir(stop_words_dir)



['StopWords_Auditor.txt',
 'StopWords_Currencies.txt',
 'StopWords_DatesandNumbers.txt',
 'StopWords_Generic.txt',
 'StopWords_GenericLong.txt',
 'StopWords_Geographic.txt',
 'StopWords_Names.txt']

In [2]:
stop_words = list()
for file in stop_word_files:
    f = os.path.join(stop_words_dir, file)
    with open(f,'r') as file_content:
        stop_words+=file_content.readlines()

In [3]:
for i in range(len(stop_words)):
    stop_words[i] = stop_words[i].replace('\n','').replace(' ', '').lower()
    if '|' in stop_words[i]:
        temp = stop_words[i]
        temp = temp.split('|')
        stop_words[i] = temp[0]
        stop_words.append(temp[1])

In [4]:
negative_words_dir = "E:\Projects\Article Text NLP Analysis\MasterDictionary\\negative-words.txt"

positive_words_dir = "E:\Projects\Article Text NLP Analysis\MasterDictionary\positive-words.txt"

positive_words_list = list()
negative_words_list = list()

with open(negative_words_dir,'r') as f:
    negative_words_list+=f.readlines()
    
for i in range(len(negative_words_list)):
    negative_words_list[i] = negative_words_list[i].replace('\n','').replace(' ', '').lower()
    
with open(positive_words_dir,'r') as f:
    positive_words_list+=f.readlines()
    
for i in range(len(positive_words_list)):
    positive_words_list[i] = positive_words_list[i].replace('\n','').replace(' ', '').lower()
    


In [44]:
import nltk
import string


def get_tokens(input_text):
    # Tokenize the input text
    tokens = nltk.word_tokenize(input_text.lower())

    # Remove stop words from the tokenized text
    filtered_tokens = [token for token in tokens if token not in stop_words]

#     # Join the filtered tokens back into a string
#     filtered_text = " ".join(filtered_tokens).translate(str.maketrans('', '', string.punctuation))
    return filtered_tokens




In [45]:
id_articles_dict = dict()

articles_dir = "E:\Projects\Article Text NLP Analysis\\article_files"
article_files = os.listdir(articles_dir)

for file in article_files:
    id = file[:-4]
    file = os.path.join(articles_dir, file) 
    with open(file, 'r',encoding = "utf-8") as f:
        article = f.read()
    id_articles_dict[id] = article


In [76]:
id_tokens_dict = dict()

token_count_dict = dict()

for id, article in id_articles_dict.items():
    id_tokens_dict[id] = get_tokens(article)
    token_count_dict[id] = len(id_tokens_dict[id])
    


In [56]:
length = len(id_tokens_dict)

In [47]:
positive_score_dict = dict()

for id, tokens in id_tokens_dict.items():
    positive_score_dict[id] = 0
    for token in tokens:
        if token in positive_words_list:
            positive_score_dict[id] +=1
            


In [50]:
negative_score_dict = dict()

for id, tokens in id_tokens_dict.items():
    negative_score_dict[id] = 0
    for token in tokens:
        if token in negative_words_list:
            negative_score_dict[id] +=1

In [59]:
polarity_score_dict = dict()

subjectivity_score_dict = dict()

for id in id_tokens_dict.keys():
    p = positive_score_dict[id]
    n = negative_score_dict[id]
    total = len(id_tokens_dict[id])
    polarity_score_dict[id] = (p-n)/(p+n+0.000001)
    subjectivity_score_dict[id] = (p+n)/(total+0.000001)
    


In [61]:
avg_sentence_length_dict = dict()
tokenizer = nltk.tokenize.sent_tokenize
for id, article in id_articles_dict.items():
    sentences = tokenizer(article)
    sen_count = len(sentences)
    words = len(article.split())
    avg_sentence_length_dict[id] = words/sen_count


In [73]:
import re
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict

# Load the CMU pronunciation dictionary
cmu_dict = cmudict.dict()

# Function to check if a word is complex
def is_complex(word):
    num_syllables = 0
    # Use regular expressions to remove non-alphabetic characters
    word = re.sub(r'[^a-zA-Z]', '', word)
    # Use the CMU pronunciation dictionary to get the number of syllables in the word
    if word.lower() in cmu_dict:
        num_syllables = len(list(y for y in cmu_dict[word.lower()][0] if y[-1].isdigit()))
    return num_syllables >= 3


complex_word_percent_dict = dict()

complex_word_count_dict = dict()

for id, article in id_articles_dict.items():
    words = article.split()
    # Create a list of complex words
    complex_words = [word for word in words if is_complex(word)]

    # Get the number of complex words
    num_complex_words = len(complex_words)
    complex_word_count_dict[id] = num_complex_words
    complex_word_percent_dict[id] = num_complex_words*100/len(words)
    


[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\Rudrransh\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [71]:
fog_index_dict = dict()

for id,article in id_articles_dict.items():
    fog_index_dict[id] = 0.4 * (avg_sentence_length_dict[id] + complex_word_percent_dict[id])
    


In [72]:
avg_words_per_sentence = avg_sentence_length_dict

In [79]:


# function to count the number of syllables in a word
def count_syllables(word):
    vowels = "aeiouy"
    num_vowels = 0
    prev_char = None
    for char in word:
        if char.lower() in vowels:
            # don't count diphthongs and triphthongs
            if prev_char and prev_char.lower() in vowels:
                continue
            num_vowels += 1
        prev_char = char
    return num_vowels

syllable_count_per_word = dict()


for id, article in id_articles_dict.items():
    words = article.split()
    num_syllables = 0
    for word in words:
        num_syllables = sum(count_syllables(word) for word in words)
    syllable_count_per_word[id] = num_syllables/len(words)


In [85]:
# define the personal pronouns
personal_pronouns = ['I', 'we','my', 'ours', 'us','My','We','Ours','Us']
pronoun_count = dict()
for id, article in id_articles_dict.items():
    words = article.split()
    # count the occurrence of each word
    counts = {word: words.count(word) for word in personal_pronouns}
    # calculate the total occurrence of all words
    pronoun_count[id] = sum(counts.values())


In [87]:
avg_word_length = dict()
for id, article in id_articles_dict.items():
    words = article.split()
    total_characters = sum(len(word) for word in words)
    # count the total number of words
    total_words = len(words)
    # calculate the average word length
    avg_word_length[id] = total_characters / total_words

In [115]:
output_df = pd.DataFrame()

output_df['URL_ID'] = df['URL_ID']
output_df['URL'] = df['URL']


In [117]:
i=0
for url in output_df['URL_ID']:
    output_df.loc[i,'POSITIVE SCORE'] = positive_score_dict[str(url)]
    output_df.loc[i,'NEGATIVE SCORE'] = negative_score_dict[str(url)]
    output_df.loc[i,'POLARITY SCORE'] = polarity_score_dict[str(url)]
    output_df.loc[i,'SUBJECTIVITY SCORE'] = subjectivity_score_dict[str(url)]
    output_df.loc[i,'AVG SENTENCE LENGTH'] = avg_sentence_length_dict[str(url)]
    output_df.loc[i,'PERCENTAGE OF COMPLEX WORDS'] = complex_word_percent_dict[str(url)]
    output_df.loc[i,'FOG INDEX'] = fog_index_dict[str(url)]
    output_df.loc[i,'AVG NUMBER OF WORDS PER SENTENCE'] = avg_words_per_sentence[str(url)]
    output_df.loc[i,'COMPLEX WORD COUNT'] = complex_word_count_dict[str(url)]
    output_df.loc[i,'WORD COUNT'] = token_count_dict[str(url)]
    output_df.loc[i,'SYLLABLE PER WORD'] = syllable_count_per_word[str(url)]
    output_df.loc[i,'PERSONAL PRONOUNS'] = pronoun_count[str(url)]
    output_df.loc[i,'AVG WORD LENGTH'] = avg_word_length[str(url)]
    i+=1

In [118]:
output_df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,64.0,32.0,0.333333,0.081702,23.337662,22.426266,18.305571,23.337662,403.0,1175.0,1.999444,0.0,5.798553
1,38,https://insights.blackcoffer.com/what-if-the-c...,58.0,38.0,0.208333,0.124031,17.750000,14.718310,12.987324,17.750000,209.0,774.0,1.717606,6.0,5.002817
2,39,https://insights.blackcoffer.com/what-jobs-wil...,65.0,37.0,0.274510,0.098837,20.117647,21.169591,16.514895,20.117647,362.0,1032.0,1.961988,2.0,5.532164
3,40,https://insights.blackcoffer.com/will-machine-...,65.0,28.0,0.397849,0.113553,17.473684,15.301205,13.109956,17.473684,254.0,819.0,1.743373,17.0,4.913253
4,41,https://insights.blackcoffer.com/will-ai-repla...,57.0,25.0,0.390244,0.082412,22.113924,17.801946,15.966348,22.113924,311.0,995.0,1.799657,15.0,5.240985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,22.0,27.0,-0.102041,0.093333,18.693878,18.013100,14.682791,18.693878,165.0,525.0,1.858079,9.0,5.497817
110,147,https://insights.blackcoffer.com/the-future-of...,37.0,10.0,0.574468,0.050053,25.096774,17.287918,16.953877,25.096774,269.0,939.0,1.790488,2.0,5.303985
111,148,https://insights.blackcoffer.com/big-data-anal...,28.0,46.0,-0.243243,0.102351,17.590909,21.016365,15.442910,17.590909,244.0,723.0,1.858742,2.0,5.223945
112,149,https://insights.blackcoffer.com/business-anal...,35.0,4.0,0.794872,0.087054,24.896552,24.099723,19.598510,24.896552,174.0,448.0,2.044321,0.0,5.818560


In [120]:
output_df.to_excel("output.xlsx")

In [121]:
positive_score_dict


{'100': 22,
 '101': 3,
 '102': 24,
 '103': 6,
 '104': 26,
 '105': 22,
 '106': 5,
 '107': 12,
 '108': 32,
 '109': 25,
 '110': 6,
 '111': 14,
 '112': 22,
 '113': 22,
 '114': 44,
 '115': 23,
 '116': 23,
 '117': 36,
 '118': 28,
 '119': 8,
 '120': 44,
 '121': 32,
 '122': 29,
 '123': 38,
 '124': 47,
 '125': 53,
 '126': 15,
 '127': 41,
 '128': 29,
 '129': 5,
 '130': 21,
 '131': 86,
 '132': 26,
 '133': 49,
 '134': 10,
 '135': 5,
 '136': 0,
 '137': 4,
 '138': 5,
 '139': 41,
 '140': 15,
 '141': 8,
 '142': 36,
 '143': 5,
 '144': 0,
 '145': 15,
 '146': 22,
 '147': 37,
 '148': 28,
 '149': 35,
 '150': 31,
 '37': 64,
 '38': 58,
 '39': 65,
 '40': 65,
 '41': 57,
 '42': 43,
 '43': 23,
 '44': 0,
 '45': 32,
 '46': 63,
 '47': 48,
 '48': 33,
 '49': 29,
 '50': 58,
 '51': 67,
 '52': 24,
 '53': 74,
 '54': 23,
 '55': 14,
 '56': 3,
 '57': 0,
 '58': 23,
 '59': 16,
 '60': 57,
 '61': 41,
 '62': 5,
 '63': 26,
 '64': 24,
 '65': 14,
 '66': 4,
 '67': 20,
 '68': 5,
 '69': 48,
 '70': 7,
 '71': 21,
 '72': 26,
 '73': 42,
 

In [122]:
negative_score_dict


{'100': 45,
 '101': 3,
 '102': 43,
 '103': 24,
 '104': 54,
 '105': 35,
 '106': 3,
 '107': 3,
 '108': 58,
 '109': 25,
 '110': 39,
 '111': 33,
 '112': 52,
 '113': 36,
 '114': 48,
 '115': 44,
 '116': 25,
 '117': 78,
 '118': 55,
 '119': 14,
 '120': 92,
 '121': 64,
 '122': 63,
 '123': 82,
 '124': 74,
 '125': 18,
 '126': 25,
 '127': 94,
 '128': 46,
 '129': 29,
 '130': 62,
 '131': 32,
 '132': 22,
 '133': 32,
 '134': 14,
 '135': 7,
 '136': 16,
 '137': 5,
 '138': 11,
 '139': 23,
 '140': 54,
 '141': 48,
 '142': 70,
 '143': 24,
 '144': 0,
 '145': 16,
 '146': 27,
 '147': 10,
 '148': 46,
 '149': 4,
 '150': 38,
 '37': 32,
 '38': 38,
 '39': 37,
 '40': 28,
 '41': 25,
 '42': 22,
 '43': 11,
 '44': 0,
 '45': 13,
 '46': 33,
 '47': 57,
 '48': 22,
 '49': 29,
 '50': 24,
 '51': 21,
 '52': 0,
 '53': 39,
 '54': 0,
 '55': 5,
 '56': 4,
 '57': 0,
 '58': 69,
 '59': 15,
 '60': 38,
 '61': 3,
 '62': 0,
 '63': 9,
 '64': 8,
 '65': 9,
 '66': 0,
 '67': 37,
 '68': 2,
 '69': 18,
 '70': 18,
 '71': 37,
 '72': 49,
 '73': 23,
 

In [123]:
polarity_score_dict


{'100': -0.34328357696591677,
 '101': 0.0,
 '102': -0.2835820853196704,
 '103': -0.5999999800000007,
 '104': -0.34999999562500006,
 '105': -0.2280701714373654,
 '106': 0.24999996875000394,
 '107': 0.5999999600000027,
 '108': -0.2888888856790124,
 '109': 0.0,
 '110': -0.7333333170370374,
 '111': -0.40425531054775937,
 '112': -0.40540539992695407,
 '113': -0.24137930618311543,
 '114': -0.04347826039697543,
 '115': -0.3134328311427936,
 '116': -0.041666665798611134,
 '117': -0.36842104939981535,
 '118': -0.32530120089998554,
 '119': -0.27272726033057904,
 '120': -0.35294117387543256,
 '121': -0.33333332986111114,
 '122': -0.36956521337429116,
 '123': -0.36666666361111117,
 '124': -0.22314049402363229,
 '125': 0.4929577395358065,
 '126': -0.24999999375000018,
 '127': -0.39259258968449934,
 '128': -0.2266666636444445,
 '129': -0.7058823321799315,
 '130': -0.49397589766294103,
 '131': 0.4576271147658719,
 '132': 0.08333333159722227,
 '133': 0.20987654061880814,
 '134': -0.1666666597222225,
 

In [124]:
subjectivity_score_dict


{'100': 0.08082026528248461,
 '101': 0.06593406520951577,
 '102': 0.10983606539371137,
 '103': 0.0714285712585034,
 '104': 0.12480499200498442,
 '105': 0.10836501880538969,
 '106': 0.031496062868125736,
 '107': 0.04297994257025804,
 '108': 0.13657056124951356,
 '109': 0.162337661810592,
 '110': 0.07317073158834027,
 '111': 0.08020477802012836,
 '112': 0.11419753068796677,
 '113': 0.0776439088652692,
 '114': 0.10861865394496027,
 '115': 0.06215213352304996,
 '116': 0.12765957412856496,
 '117': 0.0979381442457576,
 '118': 0.12848297193733285,
 '119': 0.09565217349716447,
 '120': 0.10240963847710116,
 '121': 0.14656488527242004,
 '122': 0.09368635428341512,
 '123': 0.12219959254358494,
 '124': 0.0985342018741578,
 '125': 0.0681381957119595,
 '126': 0.08130081284288453,
 '127': 0.14062499985351562,
 '128': 0.1425855510597233,
 '129': 0.07906976725797729,
 '130': 0.0835850955855135,
 '131': 0.10855565767382183,
 '132': 0.0587515299158488,
 '133': 0.12558139515413738,
 '134': 0.0895522384718

In [125]:
avg_sentence_length_dict


{'100': 21.74576271186441,
 '101': 17.545454545454547,
 '102': 18.466666666666665,
 '103': 18.37142857142857,
 '104': 22.66,
 '105': 28.657894736842106,
 '106': 32.833333333333336,
 '107': 19.967741935483872,
 '108': 32.6875,
 '109': 23.774193548387096,
 '110': 22.29787234042553,
 '111': 19.884615384615383,
 '112': 19.30188679245283,
 '113': 23.09090909090909,
 '114': 22.442857142857143,
 '115': 24.77027027027027,
 '116': 80.54545454545455,
 '117': 27.65714285714286,
 '118': 16.0125,
 '119': 21.5625,
 '120': 24.431818181818183,
 '121': 25.15625,
 '122': 20.023255813953487,
 '123': 24.213333333333335,
 '124': 29.53030303030303,
 '125': 28.6875,
 '126': 25.696969696969695,
 '127': 24.692307692307693,
 '128': 22.022222222222222,
 '129': 22.733333333333334,
 '130': 33.58,
 '131': 24.957142857142856,
 '132': 23.033898305084747,
 '133': 24.390243902439025,
 '134': 26.642857142857142,
 '135': 24.071428571428573,
 '136': 20.3,
 '137': 33.857142857142854,
 '138': 18.5,
 '139': 18.14705882352941

In [126]:
complex_word_percent_dict


{'100': 17.537022603273577,
 '101': 12.435233160621761,
 '102': 15.252707581227437,
 '103': 15.707620528771384,
 '104': 20.476610767872902,
 '105': 14.416896235078053,
 '106': 24.365482233502537,
 '107': 11.631663974151857,
 '108': 18.64244741873805,
 '109': 8.005427408412483,
 '110': 18.22519083969466,
 '111': 19.148936170212767,
 '112': 16.911045943304007,
 '113': 17.79527559055118,
 '114': 15.404201145767027,
 '115': 18.71249318057829,
 '116': 7.6749435665914225,
 '117': 16.52892561983471,
 '118': 9.523809523809524,
 '119': 24.057971014492754,
 '120': 18.74418604651163,
 '121': 10.745341614906833,
 '122': 15.272938443670151,
 '123': 16.024229074889867,
 '124': 19.85633658286301,
 '125': 18.899782135076254,
 '126': 17.21698113207547,
 '127': 17.570093457943926,
 '128': 14.228052472250253,
 '129': 18.475073313782993,
 '130': 15.128052412150089,
 '131': 18.775042930738408,
 '132': 17.512877115526123,
 '133': 28.3,
 '134': 23.324396782841823,
 '135': 8.308605341246292,
 '136': 13.300492

In [127]:
fog_index_dict


{'100': 15.713114126055196,
 '101': 11.992275082430524,
 '102': 13.487749699157641,
 '103': 13.631619640079983,
 '104': 17.25464430714916,
 '105': 17.229916388768064,
 '106': 22.879526226734352,
 '107': 12.639762363854292,
 '108': 20.53197896749522,
 '109': 12.711848382719833,
 '110': 16.209225272048077,
 '111': 15.613420621931262,
 '112': 14.485173094302736,
 '113': 16.35447387258411,
 '114': 15.138823315449669,
 '115': 17.393105380339424,
 '116': 35.28815924481839,
 '117': 17.674427390791028,
 '118': 10.21452380952381,
 '119': 18.2481884057971,
 '120': 17.270401691331926,
 '121': 14.360636645962735,
 '122': 14.118477703049457,
 '123': 16.09502496328928,
 '124': 19.754655845266416,
 '125': 19.0349128540305,
 '126': 17.165580331618067,
 '127': 16.90496046010065,
 '128': 14.50010987778899,
 '129': 16.48336265884653,
 '130': 19.48322096486004,
 '131': 17.492874315152505,
 '132': 16.21871016824435,
 '133': 21.07609756097561,
 '134': 19.98690157027959,
 '135': 12.952013565069945,
 '136': 1

In [128]:
avg_words_per_sentence


{'100': 21.74576271186441,
 '101': 17.545454545454547,
 '102': 18.466666666666665,
 '103': 18.37142857142857,
 '104': 22.66,
 '105': 28.657894736842106,
 '106': 32.833333333333336,
 '107': 19.967741935483872,
 '108': 32.6875,
 '109': 23.774193548387096,
 '110': 22.29787234042553,
 '111': 19.884615384615383,
 '112': 19.30188679245283,
 '113': 23.09090909090909,
 '114': 22.442857142857143,
 '115': 24.77027027027027,
 '116': 80.54545454545455,
 '117': 27.65714285714286,
 '118': 16.0125,
 '119': 21.5625,
 '120': 24.431818181818183,
 '121': 25.15625,
 '122': 20.023255813953487,
 '123': 24.213333333333335,
 '124': 29.53030303030303,
 '125': 28.6875,
 '126': 25.696969696969695,
 '127': 24.692307692307693,
 '128': 22.022222222222222,
 '129': 22.733333333333334,
 '130': 33.58,
 '131': 24.957142857142856,
 '132': 23.033898305084747,
 '133': 24.390243902439025,
 '134': 26.642857142857142,
 '135': 24.071428571428573,
 '136': 20.3,
 '137': 33.857142857142854,
 '138': 18.5,
 '139': 18.14705882352941

In [129]:
complex_word_count_dict


{'100': 225,
 '101': 24,
 '102': 169,
 '103': 101,
 '104': 232,
 '105': 157,
 '106': 96,
 '107': 72,
 '108': 195,
 '109': 59,
 '110': 191,
 '111': 198,
 '112': 173,
 '113': 226,
 '114': 242,
 '115': 343,
 '116': 68,
 '117': 320,
 '118': 122,
 '119': 83,
 '120': 403,
 '121': 173,
 '122': 263,
 '123': 291,
 '124': 387,
 '125': 347,
 '126': 146,
 '127': 282,
 '128': 141,
 '129': 126,
 '130': 254,
 '131': 328,
 '132': 238,
 '133': 283,
 '134': 87,
 '135': 28,
 '136': 54,
 '137': 31,
 '138': 50,
 '139': 161,
 '140': 181,
 '141': 142,
 '142': 372,
 '143': 92,
 '144': 3,
 '145': 138,
 '146': 165,
 '147': 269,
 '148': 244,
 '149': 174,
 '150': 190,
 '37': 403,
 '38': 209,
 '39': 362,
 '40': 254,
 '41': 311,
 '42': 209,
 '43': 128,
 '44': 2,
 '45': 90,
 '46': 368,
 '47': 323,
 '48': 259,
 '49': 205,
 '50': 237,
 '51': 302,
 '52': 114,
 '53': 268,
 '54': 212,
 '55': 192,
 '56': 64,
 '57': 1,
 '58': 237,
 '59': 130,
 '60': 323,
 '61': 124,
 '62': 48,
 '63': 163,
 '64': 142,
 '65': 68,
 '66': 36,


In [130]:
token_count_dict


{'100': 829,
 '101': 91,
 '102': 610,
 '103': 420,
 '104': 641,
 '105': 526,
 '106': 254,
 '107': 349,
 '108': 659,
 '109': 308,
 '110': 615,
 '111': 586,
 '112': 648,
 '113': 747,
 '114': 847,
 '115': 1078,
 '116': 376,
 '117': 1164,
 '118': 646,
 '119': 230,
 '120': 1328,
 '121': 655,
 '122': 982,
 '123': 982,
 '124': 1228,
 '125': 1042,
 '126': 492,
 '127': 960,
 '128': 526,
 '129': 430,
 '130': 993,
 '131': 1087,
 '132': 817,
 '133': 645,
 '134': 268,
 '135': 168,
 '136': 202,
 '137': 140,
 '138': 139,
 '139': 419,
 '140': 589,
 '141': 524,
 '142': 1025,
 '143': 346,
 '144': 4,
 '145': 474,
 '146': 525,
 '147': 939,
 '148': 723,
 '149': 448,
 '150': 601,
 '37': 1175,
 '38': 774,
 '39': 1032,
 '40': 819,
 '41': 995,
 '42': 742,
 '43': 440,
 '44': 5,
 '45': 401,
 '46': 1156,
 '47': 1323,
 '48': 698,
 '49': 769,
 '50': 795,
 '51': 937,
 '52': 311,
 '53': 756,
 '54': 569,
 '55': 461,
 '56': 183,
 '57': 5,
 '58': 798,
 '59': 399,
 '60': 1001,
 '61': 408,
 '62': 228,
 '63': 671,
 '64': 4

In [131]:
syllable_count_per_word


{'100': 1.8246297739672643,
 '101': 1.5595854922279793,
 '102': 1.7057761732851986,
 '103': 1.7231726283048212,
 '104': 1.8623124448367168,
 '105': 1.6464646464646464,
 '106': 1.9365482233502538,
 '107': 1.6558966074313408,
 '108': 1.8613766730401529,
 '109': 1.5115332428765265,
 '110': 1.797709923664122,
 '111': 1.8646034816247583,
 '112': 1.8504398826979471,
 '113': 1.7874015748031495,
 '114': 1.7052832590706557,
 '115': 1.7506819421713038,
 '116': 1.4966139954853273,
 '117': 1.7897727272727273,
 '118': 1.6073380171740828,
 '119': 1.901449275362319,
 '120': 1.7939534883720931,
 '121': 1.5708074534161491,
 '122': 1.7502903600464577,
 '123': 1.7433920704845816,
 '124': 1.8081067213955875,
 '125': 1.8458605664488017,
 '126': 1.7653301886792452,
 '127': 1.8504672897196262,
 '128': 1.746720484359233,
 '129': 1.8592375366568914,
 '130': 1.8350208457415127,
 '131': 1.8586147681740126,
 '132': 1.8300220750551877,
 '133': 2.117,
 '134': 2.0214477211796247,
 '135': 1.6379821958456973,
 '136': 

In [132]:
pronoun_count


{'100': 3,
 '101': 2,
 '102': 13,
 '103': 3,
 '104': 2,
 '105': 5,
 '106': 0,
 '107': 4,
 '108': 1,
 '109': 39,
 '110': 0,
 '111': 0,
 '112': 1,
 '113': 7,
 '114': 2,
 '115': 1,
 '116': 41,
 '117': 11,
 '118': 7,
 '119': 1,
 '120': 1,
 '121': 35,
 '122': 10,
 '123': 11,
 '124': 8,
 '125': 2,
 '126': 3,
 '127': 6,
 '128': 4,
 '129': 1,
 '130': 2,
 '131': 0,
 '132': 5,
 '133': 0,
 '134': 4,
 '135': 4,
 '136': 5,
 '137': 3,
 '138': 7,
 '139': 2,
 '140': 0,
 '141': 4,
 '142': 0,
 '143': 1,
 '144': 0,
 '145': 2,
 '146': 9,
 '147': 2,
 '148': 2,
 '149': 0,
 '150': 8,
 '37': 0,
 '38': 6,
 '39': 2,
 '40': 17,
 '41': 15,
 '42': 16,
 '43': 7,
 '44': 0,
 '45': 0,
 '46': 8,
 '47': 0,
 '48': 9,
 '49': 6,
 '50': 16,
 '51': 9,
 '52': 0,
 '53': 14,
 '54': 0,
 '55': 0,
 '56': 2,
 '57': 0,
 '58': 2,
 '59': 3,
 '60': 10,
 '61': 4,
 '62': 0,
 '63': 4,
 '64': 1,
 '65': 0,
 '66': 2,
 '67': 16,
 '68': 1,
 '69': 0,
 '70': 9,
 '71': 7,
 '72': 15,
 '73': 19,
 '74': 15,
 '75': 1,
 '76': 4,
 '77': 8,
 '78': 7,
 '

In [133]:
avg_word_length

{'100': 5.395946999220577,
 '101': 4.652849740932642,
 '102': 4.932310469314079,
 '103': 5.1648522550544325,
 '104': 5.360105913503972,
 '105': 4.8292011019283745,
 '106': 5.758883248730965,
 '107': 4.856219709208401,
 '108': 5.4703632887189295,
 '109': 4.305291723202171,
 '110': 5.363549618320611,
 '111': 5.418762088974855,
 '112': 5.471163245356793,
 '113': 5.167716535433071,
 '114': 4.9630808402291535,
 '115': 5.14620840152755,
 '116': 4.322799097065463,
 '117': 5.340909090909091,
 '118': 4.6510538641686185,
 '119': 5.58840579710145,
 '120': 5.343255813953489,
 '121': 4.553416149068323,
 '122': 5.137630662020906,
 '123': 5.117290748898679,
 '124': 5.423293996921498,
 '125': 5.313725490196078,
 '126': 5.193396226415095,
 '127': 5.368847352024922,
 '128': 5.048435923309788,
 '129': 5.476539589442815,
 '130': 5.331745086360929,
 '131': 5.3989696622781915,
 '132': 5.42457689477557,
 '133': 5.962,
 '134': 5.8525469168900806,
 '135': 4.718100890207715,
 '136': 4.9088669950738915,
 '137': 